In [5]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px


In [6]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

spacex_df[spacex_df['class']==1].head()


,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
17,17,19,CCAFS LC-40,Failure (in flight),1,1952.0,F9 v1.1 B1018,v1.1
18,18,20,CCAFS LC-40,Success,1,2034.0,F9 FT B1019,FT
20,20,23,CCAFS LC-40,Success,1,3136.0,F9 FT B1021.1,FT
21,21,24,CCAFS LC-40,Success,1,4696.0,F9 FT B1022,FT
22,22,25,CCAFS LC-40,Success,1,3100.0,F9 FT B1023.1,FT


In [7]:
filtered_df_payload=spacex_df[(spacex_df['Payload Mass (kg)'] > 1000) & (spacex_df['Payload Mass (kg)'] < 2000)]
filtered_df_payload.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
8,8,10,CCAFS LC-40,Success,0,1316.0,F9 v1.1,v1.1
15,15,17,CCAFS LC-40,Success,0,1898.0,F9 v1.1 B1015,v1.1
17,17,19,CCAFS LC-40,Failure (in flight),1,1952.0,F9 v1.1 B1018,v1.1


In [8]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                   dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                    ],
                                    value='ALL',
                                    placeholder="All Sites",
                                    searchable=False
                                ),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0, max=max_payload, step=1000,value=[min_payload, max_payload]),
        

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id="success-pie-chart", component_property="figure"), 
    Input(component_id="site-dropdown", component_property="value"))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site].groupby(['Launch Site','class']).count().reset_index()
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Launch Sites success pct. of all launches')
        return fig
    else:
        fig = px.pie(filtered_df, values="Mission Outcome", 
        names='class', 
        title=f'Success and failure pct. of {entered_site}')
        return fig
            
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id="success-payload-scatter-chart", component_property="figure"), 
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,payload_range):
    if entered_site == 'ALL':
        filtered_df_payload=spacex_df[(spacex_df['Payload Mass (kg)'] > payload_range[0]) & (spacex_df['Payload Mass (kg)'] < payload_range[1])]
        fig = px.scatter(filtered_df_payload,
        x='Payload Mass (kg)',
        y='class', 
        color="Booster Version Category",
        title='Payload Mass and success')
        return fig
    else:
        filtered_df_w_payload = spacex_df[(spacex_df['Payload Mass (kg)'] > payload_range[0]) & (spacex_df['Payload Mass (kg)'] < payload_range[1])]
        filtered_df_w_payload = filtered_df_w_payload[filtered_df_w_payload['Launch Site']== entered_site]
        fig = px.scatter(filtered_df_w_payload,x='Payload Mass (kg)',y='class', 
        color="Booster Version Category",
        title='Payload Mass and success for {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Nov/2021 14:54:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2021 14:54:57] "POST /_dash-upd